In [7]:
# ==============================================================================
# PART 0: SETUP
# ==============================================================================
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np
import joblib

In [2]:
# ==============================================================================
# PART 1: DATA LOADING AND PREPROCESSING
# ==============================================================================
print("--- PART 1: Loading and Preprocessing Initial Data ---")

code_columns = {
    'ICD9_DGNS_CD_1': str, 'ICD9_DGNS_CD_2': str, 'ICD9_DGNS_CD_3': str,
    'ICD9_DGNS_CD_4': str, 'ICD9_DGNS_CD_5': str, 'ICD9_DGNS_CD_6': str,
    'ICD9_DGNS_CD_7': str, 'ICD9_DGNS_CD_8': str, 'ICD9_DGNS_CD_9': str,
    'ICD9_DGNS_CD_10': str, 'ADMTNG_ICD9_DGNS_CD': str, 'CLM_DRG_CD': str,
    'ICD9_PRCDR_CD_1': str, 'ICD9_PRCDR_CD_2': str, 'ICD9_PRCDR_CD_3': str,
    'ICD9_PRCDR_CD_4': str, 'ICD9_PRCDR_CD_5': str, 'ICD9_PRCDR_CD_6': str
}

beneficiary_2008 = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv")
beneficiary_2009 = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv")
beneficiary_2010 = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv")

chunk_size = 100000
    
inpatient_agg_list, inpatient_codes_list, inpatient_readmission_list = [], [], []
inpatient_iterator = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv", dtype=code_columns, chunksize=chunk_size)
for chunk in inpatient_iterator:
    inpatient_agg_list.append(chunk.groupby('DESYNPUF_ID').agg(Inpatient_Claim_Count=('CLM_ID', 'count'), Total_Inpatient_Payments=('CLM_PMT_AMT', 'sum')))
    inpatient_codes_list.append(chunk[['DESYNPUF_ID', 'ICD9_DGNS_CD_1']])
    chunk['CLM_ADMSN_DT'] = pd.to_datetime(chunk['CLM_ADMSN_DT'], format='%Y%m%d')
    chunk['CLM_THRU_DT'] = pd.to_datetime(chunk['CLM_THRU_DT'], format='%Y%m%d', errors='coerce')
    inpatient_readmission_list.append(chunk)

inpatient_agg = pd.concat(inpatient_agg_list).groupby(level=0).sum()
inpatient_codes = pd.concat(inpatient_codes_list)
inpatient_claims_raw = pd.concat(inpatient_readmission_list)
    
outpatient_agg_list, outpatient_codes_list = [], []
outpatient_iterator = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv", dtype=code_columns, engine='python', chunksize=chunk_size)
for chunk in outpatient_iterator:
    outpatient_agg_list.append(chunk.groupby('DESYNPUF_ID').agg(Outpatient_Claim_Count=('CLM_ID', 'count'), Total_Outpatient_Payments=('CLM_PMT_AMT', 'sum')))
    outpatient_codes_list.append(chunk[['DESYNPUF_ID', 'ICD9_DGNS_CD_1']])
        
outpatient_agg = pd.concat(outpatient_agg_list).groupby(level=0).sum()
outpatient_codes = pd.concat(outpatient_codes_list)

--- PART 1: Loading and Preprocessing Initial Data ---


In [4]:
# ==============================================================================
# PART 2: FEATURE ENGINEERING AND MERGING
# ==============================================================================
print("\n--- PART 2: Engineering Features and Merging Data ---")

all_beneficiaries = pd.concat([beneficiary_2008, beneficiary_2009, beneficiary_2010], ignore_index=True)
all_beneficiaries = all_beneficiaries.drop_duplicates(subset=['DESYNPUF_ID'], keep='last')
    
all_beneficiaries['BENE_BIRTH_DT'] = pd.to_datetime(all_beneficiaries['BENE_BIRTH_DT'], format='%m-%d-%Y')
all_beneficiaries['BENE_DEATH_DT'] = pd.to_datetime(all_beneficiaries['BENE_DEATH_DT'], format='%m-%d-%Y', errors='coerce')
reference_date = datetime(2010, 12, 31)
all_beneficiaries['Age'] = ((reference_date - all_beneficiaries['BENE_BIRTH_DT']).dt.days / 365.25).astype(int)
all_beneficiaries['Is_Dead'] = all_beneficiaries['BENE_DEATH_DT'].notna().astype(int)
chronic_condition_cols = [col for col in all_beneficiaries.columns if col.startswith('SP_')]
for col in chronic_condition_cols:
    all_beneficiaries[col] = all_beneficiaries[col].replace(2, 0)
all_beneficiaries['Chronic_Condition_Count'] = all_beneficiaries[chronic_condition_cols].sum(axis=1)
    
master_df = all_beneficiaries.merge(inpatient_agg, on='DESYNPUF_ID', how='left')
master_df = master_df.merge(outpatient_agg, on='DESYNPUF_ID', how='left')
claims_cols_to_fill = ['Inpatient_Claim_Count', 'Total_Inpatient_Payments', 'Outpatient_Claim_Count', 'Total_Outpatient_Payments']
master_df[claims_cols_to_fill] = master_df[claims_cols_to_fill].fillna(0)

inpatient_claims_raw = inpatient_claims_raw.sort_values(by=['DESYNPUF_ID', 'CLM_ADMSN_DT'])
inpatient_claims_raw['Next_Admission_Date'] = inpatient_claims_raw.groupby('DESYNPUF_ID')['CLM_ADMSN_DT'].shift(-1)
days_to_next_admission = (inpatient_claims_raw['Next_Admission_Date'] - inpatient_claims_raw['CLM_THRU_DT']).dt.days
inpatient_claims_raw['Was_Readmitted_in_30_Days'] = (days_to_next_admission <= 30).astype(int)
readmission_summary = inpatient_claims_raw.groupby('DESYNPUF_ID')['Was_Readmitted_in_30_Days'].max().reset_index()
readmission_summary = readmission_summary.rename(columns={'Was_Readmitted_in_30_Days': 'Had_30Day_Readmission_Ever'})
master_df_readmission = master_df.merge(readmission_summary, on='DESYNPUF_ID', how='left')
master_df_readmission['Had_30Day_Readmission_Ever'] = master_df_readmission['Had_30Day_Readmission_Ever'].fillna(0)
    
all_codes = pd.concat([inpatient_codes, outpatient_codes], ignore_index=True)
diagnosis_counts = all_codes.groupby('DESYNPUF_ID').size().reset_index(name='Total_Diagnosis_Count')
unique_diagnosis_counts = all_codes.groupby('DESYNPUF_ID')['ICD9_DGNS_CD_1'].nunique().reset_index(name='Unique_Diagnosis_Count')
master_df_enhanced = master_df_readmission.merge(diagnosis_counts, on='DESYNPUF_ID', how='left')
master_df_enhanced = master_df_enhanced.merge(unique_diagnosis_counts, on='DESYNPUF_ID', how='left')
master_df_enhanced[['Total_Diagnosis_Count', 'Unique_Diagnosis_Count']] = master_df_enhanced[['Total_Diagnosis_Count', 'Unique_Diagnosis_Count']].fillna(0)
categorical_cols = ['BENE_SEX_IDENT_CD', 'BENE_RACE_CD']
master_df_enhanced = pd.get_dummies(master_df_enhanced, columns=categorical_cols, drop_first=True)
    
try:
    drug_exposure = pd.read_excel("D:/Jupyter/HealthArk_data/drug_exposure.xlsx")
    person_mapping = pd.read_excel("D:/Jupyter/HealthArk_data/person.xlsx")
    person_id_map = person_mapping[['PERSON_ID', 'PERSON_SOURCE_VALUE']].rename(columns={'PERSON_SOURCE_VALUE': 'DESYNPUF_ID'})
    drug_exposure = drug_exposure.merge(person_id_map, on='PERSON_ID', how='left')
    
    if 'DESYNPUF_ID' in drug_exposure.columns:
        drug_counts = drug_exposure.groupby('DESYNPUF_ID').size().reset_index(name='Total_Drug_Count')
        unique_drug_counts = drug_exposure.groupby('DESYNPUF_ID')['DRUG_CONCEPT_ID'].nunique().reset_index(name='Unique_Drug_Count')
        avg_days_supply = drug_exposure.groupby('DESYNPUF_ID')['DAYS_SUPPLY'].mean().reset_index(name='Avg_Days_Supply')
        master_df_final = master_df_enhanced.merge(drug_counts, on='DESYNPUF_ID', how='left')
        master_df_final = master_df_final.merge(unique_drug_counts, on='DESYNPUF_ID', how='left')
        master_df_final = master_df_final.merge(avg_days_supply, on='DESYNPUF_ID', how='left')
        drug_feature_cols = ['Total_Drug_Count', 'Unique_Drug_Count', 'Avg_Days_Supply']
        master_df_final[drug_feature_cols] = master_df_final[drug_feature_cols].fillna(0)
except FileNotFoundError:
    print("Drug or Person files not found. Skipping drug features.")
    master_df_final = master_df_enhanced.copy()
    master_df_final[['Total_Drug_Count', 'Unique_Drug_Count', 'Avg_Days_Supply']] = 0



--- PART 2: Engineering Features and Merging Data ---


In [8]:
# ==============================================================================
# PART 3: FINAL MODEL TRAINING AND EVALUATION
# ==============================================================================
print("\n--- PART 3: Tuning and Training the Champion XGBoost Model ---")

y = master_df_final['Had_30Day_Readmission_Ever']
features_to_drop = ['DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_DEATH_DT', 'Had_30Day_Readmission_Ever']
X = master_df_final.drop(columns=features_to_drop)
X = X.select_dtypes(include=['number'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

print("\nStarting Final Hyperparameter Tuning... (This may take a significant amount of time)")
xgb = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [5, 7],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='f1', cv=3, verbose=2)
grid_search.fit(X_train, y_train)

print("\nGrid Search Complete!")
print(f"Best F1-score found during search: {grid_search.best_score_:.4f}")
print("Best parameters found:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_
joblib.dump(best_model, 'xgboost_champion_model.pkl')
print("\nSaved trained XGBoost model to 'xgboost_champion_model.pkl'")
y_pred = best_model.predict(X_test)

print("\n--- Final Tuned Model Performance Evaluation ---")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



--- PART 3: Tuning and Training the Champion XGBoost Model ---

Starting Final Hyperparameter Tuning... (This may take a significant amount of time)
Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=200, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=200, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=200, subsample=0.8; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=200, subsample=1.0; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=200, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=200, subsample=1.0; total time=   0.5s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=5, n_estimators=300, subsample=0

In [6]:
# ==============================================================================
# PART 4: VISUALIZATIONS AND OPTIMIZATION
# ==============================================================================
print("\n--- PART 4: Creating Visualizations and Optimizing Threshold ---")
from sklearn.metrics import precision_recall_curve, f1_score

# Get predicted probabilities
y_probs = best_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f"Best Threshold for F1-Score: {best_threshold:.4f}")
print(f"Best F1-Score at this threshold: {best_f1_score:.4f}")
final_predictions = (y_probs >= best_threshold).astype(int)

print("\n--- Final Performance Report (with Optimal Threshold) ---")
print("\nClassification Report:")
print(classification_report(y_test, final_predictions))


--- PART 4: Creating Visualizations and Optimizing Threshold ---
Best Threshold for F1-Score: 0.7130
Best F1-Score at this threshold: 0.6414

--- Final Performance Report (with Optimal Threshold) ---

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97     22021
         1.0       0.50      0.90      0.64      1250

    accuracy                           0.95     23271
   macro avg       0.75      0.92      0.81     23271
weighted avg       0.97      0.95      0.95     23271



In [9]:
master_df_final.to_csv('final_processed_data.csv', index=False)

print("\n--- Successfully saved the final processed data to 'final_processed_data.csv' ---")


--- Successfully saved the final processed data to 'final_processed_data.csv' ---
